In [2]:
# import liblary
import torch
import os
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
from src.data import get_datasetCML, data_semi_learning
from src.logging_config import logger
from src.model.model import (
    LSTMModel,
    LeNet,
    LeNet_5,
    resnet20,
    resnet32,
    resnet44,
    resnet56,
    resnet110,
    resnet1202,
)
from src.utils import *

MODEL_DICT = {
    "LSTMModel": LSTMModel,
    "LeNet": LeNet,
    "LeNet_5": LeNet_5,
    "resnet20": resnet20,
    "resnet32": resnet32,
    "resnet44": resnet44,
    "resnet56": resnet56,
    "resnet110": resnet110,
    "resnet1202": resnet1202,
}

d:\2024\FIL 2024\luan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Visualization Data
1. First train model with 5 classes of Data (Cifar10)
epoch [1, 5, 10, 20, 40, 50] --> acc increase
--> Save model parameter 

2. Visualized
Data with class (5 + n) collect outputs and visulized 

In [14]:
model_run = "LeNet_5"
data_use = "cifar10"
num_classes = 10
batch_size = 64
learning_rate = 1e-5
labeled_classed = [0,1,2,3,4]
epochs = 50
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
##
select_model = [0, 4, 9, 19, 39, 49]

In [4]:
# trainning model with 5 classes [0,1,2,3,4]
trainset_5class, trainset_unlabeled = data_semi_learning(data_use=data_use, num_class=num_classes, batch_size=64, labeled_classes=labeled_classed)

for batch in trainset_5class:
    print(f"check check the batch: {batch[0]}:\n")
    # print(f"check check the idx: {idx}")
    # print(f"check check the idx: {data}")
    break

# for batch in trainset_unlabeled:
#     print(f"check check the batch: {batch[0]}:\n")
#     # print(f"check check the idx: {idx}")
#     # print(f"check check the idx: {data}")
#     break


Files already downloaded and verified
Files already downloaded and verified
check check the batch: tensor([[[-2.4291, -2.4291, -2.4291,  ...,  1.6030,  1.6612, -2.4291],
         [ 1.5642,  1.0021,  0.9439,  ...,  1.6224,  1.1571, -2.4291],
         [ 1.6999,  1.6805,  1.6224,  ...,  1.5061,  1.0214, -2.4291],
         ...,
         [-2.4291,  0.5368,  1.5642,  ...,  2.0101,  1.6612,  2.0101],
         [-2.4291,  0.4399,  1.0214,  ...,  1.5061,  1.1571,  1.3316],
         [-2.4291,  0.3817,  0.4205,  ..., -2.4291, -2.4291, -2.4291]],

        [[-2.4183, -2.4183, -2.4183,  ...,  1.6724,  1.7314, -2.4183],
         [ 1.6331,  1.0628,  1.0038,  ...,  1.6921,  1.2201, -2.4183],
         [ 1.7708,  1.7511,  1.6921,  ...,  1.5741,  1.0824, -2.4183],
         ...,
         [-2.4183,  0.5908,  1.6331,  ...,  2.0855,  1.7314,  2.0855],
         [-2.4183,  0.4924,  1.0824,  ...,  1.5741,  1.2201,  1.3971],
         [-2.4183,  0.4334,  0.4728,  ..., -2.4183, -2.4183, -2.4183]],

        [[-2.2214

In [13]:
trainloader5classes = DataLoader(trainset_5class, batch_size=64, shuffle=True)

## trainning
def train_one_epoch(model, trainloader, optimizer, criterion, device, epoch):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}", leave=True)

    for idx, (inputs, labels) in enumerate(progress_bar):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs, _ = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(trainloader)
    accuracy = 100 * correct / total

    logger.info(f"Epoch [{epoch+1}] | Loss: {epoch_loss:.4f} | Accuracy: {accuracy:.2f}%")

def trainning_main():
    model_cls= MODEL_DICT.get(model_run, None)
    model = model_cls(num_classes=num_classes).to(device)
    
    trainloader = trainloader5classes
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        train_one_epoch(model, trainloader, optimizer, criterion, device, epoch)
        if epoch in [0, 4, 9, 19, 39, 49]:
            torch.save(model.state_dict(), f"model_save/{model_run}_{data_use}_{epoch}.pth")
            print(f"Model saved at model_save/{model_run}_{epoch}.pth")
    

trainning_main()

Epoch 1: 100%|██████████| 391/391 [00:17<00:00, 22.81it/s]
Epoch [1] | Loss: 2.1336 | Accuracy: 23.70% 
 (3329526526.py:30)


Model saved at model_save/LeNet_5_0.pth


In [17]:
def generated_feature():
    folder_model = "D:\\2025\\Projects\\self_supervisedlearning\\lstm\\model_save\\LeNet_with_5classes\\"
    file_list = [f for f in os.listdir(folder_model) if f.endswith(".pth")]
    
    print(f"Found .pth files: {file_list}")
    
    if not file_list:
        print("No .pth files found!")
        return
    
    path = f"LeNet_cifar10_{0}.pth"
    model_data = torch.load(os.path.join(folder_model, path), map_location=device)
    print(f"Loaded model type: {type(model_data)}")
    print(model_data)
    return model_data

model_with5classes = generated_feature()


Found .pth files: ['LeNet_cifar10_0.pth', 'LeNet_cifar10_19.pth', 'LeNet_cifar10_39.pth', 'LeNet_cifar10_4.pth', 'LeNet_cifar10_49.pth', 'LeNet_cifar10_9.pth']
Loaded model type: <class 'collections.OrderedDict'>
OrderedDict({'features.0.weight': tensor([[[[ 1.0764e-01, -9.7127e-02,  7.0996e-02, -7.3853e-02, -1.0007e-01],
          [-5.2506e-02,  2.1857e-02, -3.0240e-02, -8.3655e-02, -1.0853e-01],
          [-4.8249e-02, -1.0234e-01,  6.6734e-02, -1.0387e-01, -1.9594e-03],
          [-1.1889e-01, -5.0460e-03,  4.1202e-02, -5.6854e-02, -8.6374e-02],
          [-1.8547e-02, -8.9800e-02,  5.4877e-02,  3.4855e-02, -7.0144e-02]],

         [[-7.4720e-02, -1.0311e-01, -7.5854e-02,  1.4496e-04, -4.5453e-02],
          [ 1.2385e-02, -1.1012e-01,  7.4974e-02, -2.4934e-02, -3.8505e-02],
          [-1.3797e-02,  4.5269e-02,  1.6125e-03, -4.9393e-02, -4.7396e-02],
          [-2.4497e-02, -3.5167e-02,  4.1500e-02,  1.5700e-02, -7.6236e-02],
          [ 6.3691e-02,  1.8406e-02,  7.6503e-02, -8.7787e

In [ ]:
label_selected = [0,1,2,3,4,5]
data6classes, _ = data_semi_learning(data_use=data_use, num_class=num_classes, batch_size=64, labeled_classes=label_selected)


## Create Pseudo Label
1. Check with cluster algorithsm
- DBSCAN
- Affinity Propagation
-